# Decision tree Classification

## Preperation 

### Importing needed libraries

In [1]:
import sys
  
# setting path
sys.path.append('../data/')

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import preprocessing as pre

/Users/aaronsteiner/Documents/GitHub/Data_mining/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aaronsteiner/Documents/GitHub/Data_mining/src/models/../data/preprocessing.py:19: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nlt

# Grid search

### Test data loading

In [3]:
tfidf, df_train, df_test = pre.setup(rem_stop=False, do_stem=False, do_lem=True, upsample=True, do_emojis=False)


Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 200.28it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


In [4]:
df_train.head()

,label,tweet,preprocessed
30405,0,when everyone's free when you're in exam mode ...,"[when, everyones, free, when, youre, in, exam,..."
27807,0,#jacksonville rooster simulation: i want to ...,"[jacksonville, rooster, simulation, i, want, t..."
8660,0,@user just run 10kms for @user @user #loveis...,"[user, just, run, 10kms, for, user, user, love..."
19185,0,@user got the prototype for our new usb today!...,"[user, got, the, prototype, for, our, new, usb..."
10355,0,have a &amp; #healthy #fathersday. #runnerda...,"[have, a, amp, healthy, fathersday, runnerdad,..."


### Setup training function

In [5]:
def train_model(df_train: pd.DataFrame, tfidf: TfidfVectorizer):
    tree = DecisionTreeClassifier(random_state=55)

    pipe = Pipeline(steps=[('dec_tree', tree)])

    Xt_train = tfidf.transform(df_train['preprocessed'])
    y_train = df_train['label']
    
    criterion = ['gini', 'entropy']
    max_depth = [i for i in range(200, 400, 20)] #-> tried 100 - 280 but not a single model used values bellow 200
    #Some models used 280 therefore boost to 400 was tried 
    min_samples_split = [i for i in range(2, 20, 2)]
    min_samples_leaf = [i for i in range(1, 5)]
    #min_samples_leaf = [i for i in range(1, 10)] -> was tried but all models used 1 or 2
    class_weight = [None] #-> balanced yields f1 bellow .50

    parameters = dict(dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth, dec_tree__min_samples_split=min_samples_split,
                      dec_tree__min_samples_leaf=min_samples_leaf, dec_tree__class_weight=class_weight)

    dec_tree = GridSearchCV(pipe, param_grid=parameters, scoring='f1', n_jobs=-1)
    dec_tree.fit(Xt_train, y_train)

    return dec_tree.best_estimator_


### Setup testing function

In [6]:
def test_model(model, df_test: pd.DataFrame, tfidf: TfidfVectorizer):
    Xt_test = tfidf.transform(df_test['preprocessed'])
    y_test = df_test['label']
    y_pred = model.predict(Xt_test)

    predictions = []

    predictions.append(model.get_params())
    predictions.append(f"Precision: {metrics.precision_score(y_test, y_pred)}")
    predictions.append(f"Recall: {metrics.recall_score(y_test, y_pred)}")
    predictions.append(f"Accuracy: {metrics.accuracy_score(y_test, y_pred)}")
    predictions.append(f"F1: {metrics.f1_score(y_test, y_pred)}")

    return predictions

### Setup result list

In [7]:
results = []

### Only Tokenization

In [8]:
tfidf, df_train, df_test = pre.setup(rem_stop=False, do_stem=False, do_lem=False, upsample=False, do_emojis=False)
model = train_model(df_train, tfidf)
results.append("Only Tokenization \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 475.28it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


KeyboardInterrupt: 

### Remove Stopwords

In [ ]:
tfidf, df_train, df_test = pre.setup(rem_stop=True, do_stem=False, do_lem=False, upsample=False, do_emojis=False)
model = train_model(df_train, tfidf)
results.append("\n\nRemove Stopwords \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 283.21it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


### Emojis

In [ ]:
tfidf, df_train, df_test = pre.setup(rem_stop=True, do_stem=False, do_lem=False, upsample=False, do_emojis=True)
model = train_model(df_train, tfidf)
results.append("\n\nEmojis \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 358.43it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


### Stemming

In [ ]:
tfidf, df_train, df_test = pre.setup(rem_stop=True, do_stem=True, do_lem=False, upsample=False, do_emojis=True)
model = train_model(df_train, tfidf)
results.append("\n\nStemming \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 467.70it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


### Upsampling

In [3]:
tfidf, df_train, df_test = pre.setup(rem_stop=True, do_stem=True, do_lem=False, upsample=True, do_emojis=True)
model = train_model(df_train, tfidf)
results.append("\n\nUpsampling \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 160.05it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 
There is 47550 training data, of which 50.0% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


NameError: name 'train_model' is not defined

### All-but-Stemming

In [ ]:
tfidf, df_train, df_test = pre.setup(rem_stop=True, do_stem=False, do_lem=False, upsample=True, do_emojis=True)
model = train_model(df_train, tfidf)
results.append("\n\nAll-but-Stemming \n")
results.append(test_model(model, df_test, tfidf))

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 307.41it/s]


There is 25569 training data, of which 7.02% is hate speech 
There is 6393 test data, of which 7.01% is hate speech 


### Export results to file

In [ ]:
textfile = open("results_descision_tree.txt", "w")
for element in results:
    if not isinstance(element, str):
        for subelement in element:
            textfile.write(str(subelement) + "\n")
        continue
    textfile.write(str(element) + "\n")
textfile.close()

### Results

Only Tokenization 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(max_depth=240, min_samples_split=16, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(max_depth=240, min_samples_split=16, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'gini', 'dec_tree__max_depth': 240, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 16, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6869565217391305
Recall: 0.5290178571428571
Accuracy: 0.9501016737056155
F1: 0.5977301387137451


Remove Stopwords 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(max_depth=300, min_samples_split=6, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(max_depth=300, min_samples_split=6, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'gini', 'dec_tree__max_depth': 300, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 6, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6565934065934066
Recall: 0.5334821428571429
Accuracy: 0.9477553574221805
F1: 0.5886699507389163


Emojis 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(max_depth=260, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(max_depth=260, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'gini', 'dec_tree__max_depth': 260, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 2, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6565934065934066
Recall: 0.5334821428571429
Accuracy: 0.9477553574221805
F1: 0.5886699507389163


Stemming 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(max_depth=240, min_samples_split=4, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(max_depth=240, min_samples_split=4, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'gini', 'dec_tree__max_depth': 240, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 4, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6887052341597796
Recall: 0.5580357142857143
Accuracy: 0.9513530423901142
F1: 0.6165228113440198


Upsampling 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(criterion='entropy', max_depth=220, min_samples_leaf=2,
                       min_samples_split=6, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(criterion='entropy', max_depth=220, min_samples_leaf=2,
                       min_samples_split=6, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'entropy', 'dec_tree__max_depth': 220, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 2, 'dec_tree__min_samples_split': 6, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6130790190735694
Recall: 0.5022321428571429
Accuracy: 0.9429063037697482
F1: 0.5521472392638037


All-but-Stemming 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(criterion='entropy', max_depth=220, min_samples_leaf=2,
                       min_samples_split=6, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(criterion='entropy', max_depth=220, min_samples_leaf=2,
                       min_samples_split=6, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'entropy', 'dec_tree__max_depth': 220, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 2, 'dec_tree__min_samples_split': 6, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
Precision: 0.6179104477611941
Recall: 0.46205357142857145
Accuracy: 0.9422806194274989
F1: 0.5287356321839082
